In [ ]:
%env NVIDIA_API_KEY=nvapi-1234567890abcdef   # replace with your API key

In [ ]:
pip install -q python-dotenv langchain_nvidia_ai_endpoints

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# Get NVIDIA's API Key: the API key should start with "nvapi-"
if not os.getenv("NVIDIA_API_KEY"):
    load_dotenv(find_dotenv()) # read .env file
    apikey = os.getenv('NVIDIA_API_KEY')

## Working with NVIDIA NIM
คุณสามารถทดสอบ LLM model ต่างๆ ที่มีใน NVIDIA NIM ได้อย่างง่ายดาย โดยทำได้ 2 รูปแบบคือ
1. ติดตั้ง NVIDIA NIM ของตนเอง ซึ่งคุณสามารถติดตั้ง NIM ได้ทั้งบน on-premise หรือบน Cloud instance ของคุณเอง เช่น AWS EC2 ก็ได้
2. ทำการเรียกใช้ NIM โดยตรงจาก NVIDIA Playground (https://build.nvidia.com) ซึ่งเปิดให้ใช้งานจำนวนหนึ่ง

In [9]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
    # base_url="http://192.134.1.100:30644/v1", # ระบุ NIM endpoint ของคุณ, ถ้าไม่ระบุจะวิ่งไปที่ NVIDIA Playground
    model="meta/llama-3.1-8b-instruct",        # ระบุ LLM model ที่ต้องการใช้
    temperature=0.1, 
    max_tokens=1000, 
    top_p=1.0)

result = llm.invoke("What is the capital of Thailand?")
print(result.content)

The capital of Thailand is Bangkok.


In [10]:
result = llm.invoke("จงอธิบายความหมายของคำว่า ความรู้เท่าหางอึ่ง")
print(result.content)

ความรู้เท่าหางอึ่ง หมายถึง ความรู้ที่มีความลึกและกว้างขวางมากจนไม่สามารถจดจำหรือจำได้


## Stream, Batch, and Async
การ call model สามารถเรียกได้ทั้งแบบ Stream, Batch และ Async

#### การประมวลผลแบบ Batch 
โดยการจัดเตรียม prompt จำนวนหลายๆ prompt ให้อยู่ในรูปแบบ List แล้วส่งเข้ามาประมวลผลในครั้งเดียว

In [14]:
print(llm.batch(["What's 2*3?", "What's 2*6?"]))
# Or via the async API
# await llm.abatch(["What's 2*3?", "What's 2*6?"])

[AIMessage(content='2 × 3 = 6', response_metadata={'role': 'assistant', 'content': '2 × 3 = 6', 'token_usage': {'prompt_tokens': 18, 'total_tokens': 25, 'completion_tokens': 7}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'}, id='run-771db744-ec33-41fe-b0ef-e414b78f49c1-0', role='assistant'),
 AIMessage(content='2 * 6 = 12', response_metadata={'role': 'assistant', 'content': '2 * 6 = 12', 'token_usage': {'prompt_tokens': 18, 'total_tokens': 25, 'completion_tokens': 7}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'}, id='run-995d4280-9d55-4e61-9ba5-3d0e12a9795b-0', role='assistant')]

#### การประมวลผลแบบ Stream จะส่ง output ออกมาทีละ Token

In [12]:
for chunk in llm.stream("How far can a seagull fly in one day?"):
    # Show the token separations
    print(chunk.content, end="|")

|Se|ag|ulls| are| incredibly| skilled| and| long|-distance| f|liers|.| Their| flying| abilities| are| adapted| to| their| marine| environment|,| and| they| can| cover| impressive| distances| in| search| of| food|,| shelter|,| and| breeding| grounds|.

|The| distance| a| se|ag|ull| can| fly| in| one| day| depends| on| various| factors|,| such| as|:

|1|.| **|Species|**:| Different| se|ag|ull| species| have| varying| flying| abilities|.| For| example|,| the| H|erring| G|ull| (|L|arus| argent|atus|)| is| known| for| its| impressive| long|-distance| flights|,| while| the| Laugh|ing| G|ull| (|Le|uc|op|ha|eus| atr|ic|illa|)| is| more| adapted| to| coastal| areas| and| may| not| fly| as| far|.
|2|.| **|Weather| conditions|**:| Wind|,| temperature|,| and| humidity| can| significantly| impact| a| se|ag|ull|'s| flying| ability|.| Strong| head|w|inds|,| for| instance|,| can| reduce| their| flying| range|.
|3|.| **|Food| availability|**:| Se|ag|ulls| may| fly| longer| distances| to| find| food|,| 

#### การประมวลผลแบบ Async

In [17]:
async for chunk in llm.astream(
    "How long does it take for monarch butterflies to migrate?"
):
    print(chunk.content, end="|")

|The| incredible| monarch| butterfly| migration|!

|The| monarch| butterfly| migration| is| a| remarkable| phenomenon| that| takes| place| every| year|,| with| millions| of| monarch|s| traveling| from| Canada| and| the| United| States| to| Mexico| and| California|.| The| duration| of| the| migration| varies| depending| on| the| location| and| the| specific| monarch| population|,| but| here| are| some| general| guidelines|:

|**|From| Canada| and| the| United| States| to| Mexico|:|**

|*| The| monarch|s| that| migrate| from| Canada| and| the| United| States| to| Mexico| typically| begin| their| journey| in| late| summer| or| early| fall|,| around| August| or| September|.
|*| The| journey| can| take| anywhere| from| |2| to| |4| weeks|,| covering| a| distance| of| around| |3|,|000| to| |4|,|000| miles| (|4|,|800| to| |6|,|400| kilometers|).
|*| The| monarch|s| fly| at| an| average| speed| of| about| |10|-|15| miles| (|16|-|24| kilometers|)| per| hour|,| with| some| individuals| reaching| 

**เรียกดูว่าในระบบมี model อะไรให้ใช้งานบ้าง**

In [ ]:
llm.get_available_models()

### General Chat

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA

prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant named Fred."), ("user", "{input}")]
)
chain = prompt | llm | StrOutputParser()

for txt in chain.stream({"input": "What's your name?"}):
    print(txt, end="")

My name is Fred, and I'm a helpful AI assistant. I'm here to assist you with any questions or tasks you may have. What can I help you with today?

คุณอาจจะทดสองเปลี่ยนชื่อดู จะเห็นว่าตัว Chat สามารถจดจำชื่อคุณได้
อย่างไรก็ตาม การเรียก Chat แบบนี้เป็นการทำงานแบบ Single turn ซึ่งจะจดจำข้อมูลได้เฉพาะภายใน prompt เดียวกันเท่านั้น

### Code Generation
ถัดไปเราจะลองเปลี่ยน model จาก Llama-3.1 มาเป็นโมเดลอีกตัว คือ codellama-70b ซึ่งใช้สำหรับ generate code

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert coding AI. Respond only in valid python; no narration whatsoever.",
        ),
        ("user", "{input}"),
    ]
)
chain = prompt | ChatNVIDIA(model="meta/codellama-70b") | StrOutputParser()

query = """give me python code to define 2 function - \
        fact(n) will compute the value of n factorial, \
        mysquare(x) will compute square of x, \
        then show show sample to use those functions.
        """

for txt in chain.stream({"input": query}):
    print(txt, end="")


def fact(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * fact(n-1)

def mysquare(x):
    return x * x

# Test the functions
print(fact(5))  # Output: 120
print(mysquare(5))  # Output: 25

In [25]:
# test code generated by model

def fact(n):
    if n == 0 or n == 1:
        return 1
    else:
        return n * fact(n-1)

def mysquare(x):
    return x * x

# Test the functions
print(fact(5))  # Output: 120
print(mysquare(5))  # Output: 25

120
25
